## 간단 스노우 어플 체험

스노우 어플과 같이 실시간으로 영상을 읽어오면서 얼굴이 있는 경우 얼굴의 스티커를 붙여보도록 합시다.   

- moviepy를 시작으로 파이썬 상에서 이미지/영상을 어떻게 처리하는지 정리하면서 만들어 봅시다.


## 1. moviepy로 비디오 처리하기

먼저 우리는 동영상을 쉽게 다룰 수 있는 방법을 찾고 있습니다. 그래서 대안으로 떠오른 것은 **`moviepy`**라는 파이썬 기반의 동영상 처리 라이브러리입니다.

하지만 우리는 동영상을 다룰 때의 실행시간, 즉 처리 속도가 중요하다는 것도 알고 있습니다. 그래서 **moviepy**를 쓰는 것이 성능면에서 적합한지 알아보려고 합니다.

**1. moviepy를 이용해서 주피터 노트북 상에서 비디오를 읽고 쓰는 프로그램을 작성**

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

샘플로 제공된 **`video2.mp4`**를 **moviepy**로 읽어서 **width=640**으로 축소하여 화면에 플레이해 보고, 플레이한 내용을 **mvpyresult.mp4**라는 파일로 저장해 보겠습니다. 저장이 완료되면 샘플 원본과 새롭게 저장된 동영상 파일을 열어서 두 영상의 화면크기나 파일 사이즈를 비교해 보세요.      

**원본 : 10.7MB**      
**샘플 :  876.3 KB**     

In [2]:
# 읽기  
video_path = os.path.join(os.getenv('HOME') , 'aiffel/video_sticker_app/images/video2.mp4' )

clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/ssac27/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/ssac27/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/ssac27/aiffel/video_sticker_app/images/mvpyresult.mp4


**2. moviepy 로 읽은 동영상을 numpy 형태로 변환하고 영상 밝기를 50% 어둡게 만든 후에 저장**

In [3]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)

# v_len은 이미지 한장이 몇개가 들어있는지 나타내니 batch_size와 같은 의미이다.
# batch_size, y, x ,3[RGB]
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  12%|█▏        | 48/403 [00:00<00:00, 476.04it/s, now=None]

Moviepy - Building video /home/ssac27/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/ssac27/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/ssac27/aiffel/video_sticker_app/images/mvpyresult2.mp4


**3. 영상을 읽고 쓰는 시간을 측정해 주세요. OpenCV를 사용할 때와 차이를 측정**


#### moviepy 사용
시간 : 2.54ms

In [7]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)
print(vlen)
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  11%|█▏        | 46/403 [00:00<00:00, 459.47it/s, now=None]

403
Moviepy - Building video /home/ssac27/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/ssac27/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/ssac27/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 2.54ms


In [10]:
vlen

403

In [9]:
clip.fps

30.0

#### OpenCV 사용
시간 : 1.49ms

In [6]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

print(vlen)


for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

405
[INFO] cv time : 1.49ms


OpenCV가 더 빠른 속도를 보이고 있습니다. 

**4. moviepy 를 이용할 때의 장단점을 분석**   

## moviepy 를 이용할 때의 장단점 분석

### movieoy를 사용할 때

- 여러 비디오에 대해 처리할 때
- 여러 비디오를 복잡한 방식으로 합칠 때
- video effect를 추가하고 싶을 때(다른 video editor 없이)
- 여러 이미지를 이용해 GIF를 만들고 싶을 때

### MoviePy 특징
- 간단하며 직관적임
- Flexible함
- Protable함
- numpy와의 호환성

### MoviePy 단점 

- Stream video에 대한 작업   [Open CV에 비해 느린속도]
   - openCV와 비교했을때 더 느린 속도의 시간을 가지니 빠른 처리가 중요한 Stream에는 OpenCV를 사용하는게 적합하다.  
   
   
- 처리하는 vldeo의 개수가 많을 경우   [ 위와 동일한 이유 ]
    - 처리 속도가 더 빠른 OpenCV를 사용하는것이 더 낫다.

#### OpenCV와 비교했을때 MoviePy의 장점 

**1. 서로 다른 video와 audio 합성이 쉽다.**

비디오 내부의 audio 데이터를 간편하게 다룰 수 있습니다.  

In [ ]:
# video에 다른 audio를 넣고 싶다면 아래의 코드를 사용하면 video
from moviepy.editor import *
videoclip = VideoFileClip("old_video.mp4").subclip(1, 10)
audioclip = AudioFileClip("new_audio.mp3").subclip(1, 10)

videoclip.audio = audioclip
videoclip.write_videofile("new_video.mp4")

audio file이나 video file에서 오디오를 추출하는 기능을 하는 class

In [23]:
from moviepy.editor import *
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
video_clip = VideoFileClip(video_path)
audioclip = video_clip.audio

# audio 데이터 추출
audioclip

**2. video frame 추출**   
movipy 내장 함수를 이용해 특정 초의 frame을 numpy array로 추출 가능합니다.

In [21]:
# get frame 함수를 이용해 특정 시간대의 frame 추출
img = clip.get_frame(10)

---

## 2. 카메라 스티커 프로그램 어디까지 만들고 싶은지 정의하기

**1. 실시간 카메라 스티커 프로그램을 만들어봅시다.**  

image에 sticker를 붙이는 **img2sticker**라는 이름의 메소드를 만들고 이를 **newaddsticker.py**에 저장하겠습니다. 


- **CASE 1 : 웹캠 입력을 사용하는 경우** 

**`cv2.VideoCapture(0)`**를 이용하면 웹캠 입력을 받아올 수 있습니다. 현재의 기본적인 **img2sticker** 를 활용하여 가장 기본적인 웹캠 실시간 스티커앱을 아래와 같이 빠르게 만들어 보겠습니다. 여기서 파라미터로 주어지는 **0**은 시스템에 연결된 영상 입력장치의 인덱스입니다. 대부분의 경우 웹캠이 하나만 달려있을 테니, **0**을 사용하면 됩니다.



``` python

import numpy as np
import cv2
import dlib

from newaddsticker import img2sticker

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)   # 연결된 영상 장치의 인덱스, 하나만 있는 경우 0을 사용
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)

        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()
   
     
```

위 코드를 **webcam_sticker.py**에 저장하였습니다. 이후 터미널에서 아래와 같이 실행해볼 수 있습니다.

```
$ cd ~/aiffel/video_sticker_app && python webcam_sticker.py
```

### RTMP(Real Time Message Protocol)

독점 프로토콜로 비디오나 오디오 등을 인터넷 상에서 실시간으로 스트리밍 데이터를 전송해서 불특정 다수들이 받아 볼 수 있도록 하는 기술의 규격입니다.  
RTMP는 기본 **1935**포트를 사용하지만 통신이 실패하면 **RTMPS(434)**나 **RTMPT(80)** 포트를 사용하여 통신하도록 시도합니다. 


- **CASE 2 : 스마트폰 영상의 스트리밍 입력을 사용하는 경우**

이번엔  스마트폰에 있는 영상촬영 어플리케이션을 통해 녹화된 동영상 스트리밍을 입력으로 해서 진행해 보겠습니다. 

동영상 스트리밍을 위한 다양한 프로토콜 중, OpenCV로 처리할 수 있는 것으로 **RTMP(Real Time Message Protocol)**를 활용하도록 하겠습니다. **RTMP**에 대한 자세한 기술적인 내용은 이번 노드의 범위를 벗어나는 내용이므로 생략하겠습니다. 이 **[링크](https://juyoung-1008.tistory.com/30)**의 글을 참고하면 **RTMP**가 다양한 동영상 스트리밍 서비스에 널리 사용되고 있음을 알수 있습니다.


스마트폰 카메라로 촬영한 영상을 RTMP로 스트리밍할 수 있게 해주는 어플리케이션은 많습니다. 그러나, RTMP 스트리밍을 위해서는 별도의 서버에서 **`rtmp:// url`**을 오픈해 주어야 합니다. 
- 사용하는 어플리케이션이 RTMP URL을 자동으로 생성해주는 경우 생성된 URL을 사용하면 됩니다.

---
**2. 스티커앱을 실행하고 카메라를 고정하고 서서히 멀어져봅니다. 혹은 아주 가까이 다가가 봅니다. 얼굴을 찾지 못하는 거리를 기록해주세요.**  

일반적으로 약 15ch ~ 1m 30cm 범위 사이에서 얼굴 인식이 가능하다고 합니다. 실제로 측정했을 때 어떠한지 결과를 기록해 주세요.


### Resolution : (640 x 480) 일때 측정 결과
    - 줄자로 직접 쟀기에 어느정도의 오차가 있습니다.
- **최소 : 약 14cm**

- **최소 : 약 111cm**

**3. 다시 자리로 돌아온 후 고개를 상하좌우로 움직여주세요. yaw, pitch, roll 각도의 개념을 직접 실험해 보고 각각 몇 도까지 정상적으로 스티커앱이 동작하는지 기록해주세요.**   
(참고)

- yaw : y축 기준 회전 → 높이 축
- picth : x축 기준 회전 → 좌우 축
- roll : z축 기준 회전 → 거리 축

일반적인 허용 범위는 아래와 같다고 알려져 있습니다.

- yaw : -45 ~ 45도
- pitch : -20 ~ 30도
- roll : -45 ~ 45도

실제 측정해 본 결과는 어떠한지 기록해 주세요.

### Resolution : (640 x 480) 일때 측정 결과
    - 줄자로 직접 쟀기에 어느정도의 오차가 있습니다.
- **yaw : 약 -50 ~ 40도**


- **pitch : 약 -45 ~ 45도**


- **roll : 약 -50 ~ 50도**

---

**4. 만들고 싶은 스티커앱의 스펙(허용 거리, 허용 인원 수, 허용 각도, 안정성)을 정해주세요.**   

(예시)

- 거리 : 25cm ~ 1m → 너무 가까우면 스티커의 의미가 없음, 셀카봉을 들었을 때의 유효거리


- 인원 수 : 4명 → 4인 가족 기준


- 허용 각도 : pitch : -20 ~ 30도, yaw : -45 ~ 45도, roll : -45 ~ 45도 → 화면을 바라볼 수 있는 각도


- 안정성 : 위 조건을 만족하면서 FPPI (false positive per image) 기준 < 0.003, MR (miss rate) < 1 300장당 1번 에러 = 10초=30*10에 1번 에러

기준의 이유를 어떻게 정했는지가 중요합니다. → 서비스 관점, 엔지니어링 관점으로 설명하면 좋습니다.

- **거리 : 30 ~ 80cm** -> 손으로 폰을 든채 셀카를 찍을때 너무 가깝게 들지 않고 거리가 떨어진 상태에서 찍었을때의 유효 거리    


- **인원 수 : 4명**  ->  5인 미만 거리두기 

- **허용 각도 pitch : -20 ~ 30도, yaw : -45 ~ 25도, roll : -30 ~ 30도**  -> 셀카를 찍으려 고개 각도를 틀었을때 적정 각도


- **안정성 :** 위 조건을 만족하면서 **FPPI (false positive per image)** 기준 < 0.003, MR (miss rate) < 1 300장당 1번 에러 = 10초=30*10에 1번 에러




---

##  3. 스티커 Out Bound 예외처리 하기
이전 스텝에서 만든 기본 웹캠 스티커앱의 문제점들을 찾아서 보완해 가도록 합시다.

**1. 지금까지 만든 스티커앱을 이용해서 예외 상황을 찾아봅니다.**   

발생하는 예외 상황을 기재하면 다음과 같습니다.



1. 얼굴이 카메라 왼쪽 경계를 벗어나서 detection 되는 경우 예외가 발생합니다.   [화면 종료 ]


2. 얼굴 detection이 되면서 sticker와 영상을 합성할 때 빠르게 얼굴을 움직여서 얼굴은 인식되는데 스티커가 못 붙이게 만들면 예외가 발생합니다.  


> 모든 예외 문구는 아래와 같습니다.   [img 합성할때 오류가 발생]

``` python
Traceback (most recent call last):
  File "webcam_sticker.py", line 44, in <module>
    main()
  File "webcam_sticker.py", line 32, in main
    img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   
  File "/home/ssac27/aiffel/video_sticker_app/newaddsticker.py", line 42, in img2sticker
    cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)
cv2.error: OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/core/src/arithm.cpp:666: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'

```    

---

**2. 문제가 어디에서 발생하는지 코드에서 확인합니다.
(힌트) 얼굴이 카메라 왼쪽 경계를 벗어나서 detection 되는 경우 refined_x 의 값이 음수가 됩니다.
img_bgr[..., refined_x:...] 에서 numpy array의 음수 index에 접근하게 되므로 예외가 발생합니다.**

newaddsticker.py의 img2sticker 메소드에서 아래 부분을 수정해 주어야 합니다.

카메라에 얼굴이 인식될때 코 부분이 왼쪽 화면 경계로 넘어가거나 근처에 다가갈때 x의 값이 0에 가깝게 되고 그때의 dlib가 인식한 rect의 w[너비] 값의 반보다 작게되면  reifined_x가 음수가되어 인덱스 범위에 오류가 발생한다.   

- dlib가 인식한 너비의 절반보다 x[코 위치] 부분의 좌표값이 작은 경우 오류가 발생한다. 

``` python
### (이전 생략) ###

# sticker
img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)

refined_x = x - w // 2
refined_y = y - h

if refined_y < 0:
    img_sticker = img_sticker[-refined_y:]
    refined_y = 0

###
# TODO : x 축 예외처리 코드 추가
###
if refined_x < 0:
    img_sticker = img_sticker[:-refined_x]
    refined_x =0
elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
    img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]


img_bgr = img_orig.copy()
sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
    cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

return img_bgr

```

---

#### 3. Out bound 오류(경계 밖으로 대상이 나가서 생기는 오류)를 해결해 주세요.
위 예외처리 코드 부분에 들어가야 하는 코드는 아래와 같습니다. **newaddsticker.py** 파일을 수정해 주세요.
``` python
    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]
        
```

---

#### 4. 다른 예외는 어떤 것들이 있는지 정의해 주세요. 어떤 것이 문제가 되는지 스스로 정해봅시다. 

꼼꼼이 찾아보면 위에서 수정한 것과 같은 명백한 오류 발생 케이스 이외에도 다양한 예외 상황을 찾아볼 수 있을 것입니다.

정확한 정답은 없습니다. 본인이 정의하기 나름입니다.
저는 '고개를 왼쪽아래로 향하게 할 때 스티커의 모양이 일정한 것' 을 예외 상황이라고 정의했습니다.

---
#### 예외 상황 :   
- **고개의 각도가 똑바르지 않은 경우**   
ex) 왼쪽 아래나 오른쪽 위 등을 바라볼 때도 스티커의 모양이 일정하게 붙어 있어서 스티커의 반은 얼굴 위 나머지 반은 공중에 떠 있는 상태인 것 


- **화면과 거리가 매우 가까운 경우** 얼굴을 제대로 인식하지 못해서 이미지를 붙이지 못한다.
  - **화면과 거리거 먼 경우**에도 위와 동일하다. 

---

## 4. 스티커앱 분석 - 거리, 인원 수, 각도, 시계열 안정성


카메라와 사람의 거리 및 각도, 카메라 상에 존재하는 사람의 수, 시계열 안정성에 따른 다양한 예외상황이 있습니다.     
위와 같은 다양한 예외상항을 찾아봅시다.  

**1. 멀어지는 경우에 왜 스티커앱이 동작하지 않는지 분석해 봅시다.**  

스티커 앱의 중요 단계는 다음과 같습니다.  [**detection**, **landmark**, **blending**]    <br>

- 얼굴을 인식[**detection**]하고 얼굴의 [**landmark**]를 찾은 후 이미지 스티커와 원본 이미지를 섞는 [**blending**]  단계 중에 dlib detection에서 문제가 발생합니다.   

- 카메라에서 사람이 멀어지는 경우 **detector_hog** 단계에서 bbox 가 출력되지 않습니다.

``` python
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 0)
    if len(dlib_rects) < 1:
        return img_orig
```

---

**2. detector_hog의 문제를 해결하기 위해, 이미지 피라미드를 조절하여 성능을 향상시키겠습니다. 위에 단계에서 만든 img2sticker 메소드를 간단히 고쳐 봅시다.**

아래의 코드를 사용해서 img pyramid 수를 조정하면 됩니다.   


``` python
def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 1) # <- 이미지 피라미드 수 변경
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb_vga, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
```

이미지 파라미드를 변경하고 실행하면 문제가 발생할 것입니다.  

**webcam_sticker.py** 를 다시한번 실행하여 스티커앱이 어떻게 동작하는지 살펴보고 문제가 생기는 경우 어떤 문제가 발생하는지 알아봅시다.    

---

#### 3. 위에서 새롭게 시도한 방법의 문제점

속도가 현저히 느려집니다. 기존 30ms/frame 에서 120ms/frame 으로 약 4배 느려짐 → 실시간 구동이 불가능 합니다.

![image_pyramid_%EC%88%98%EC%A0%95%20%ED%9B%84.png](./images/image_pyramid_%EC%88%98%EC%A0%95%20%ED%9B%84.png)

#### 4. 실행시간을 만족할 수 있는 방법을 찾아봅시다.

- hog 디텍터를 딥러닝 기반 디텍터로 변경할 수 있습니다. hog 학습 단계에서 다양한 각도에 대한 hog 특징을 모두 추출해서 일반화 하기 어렵기 때문에 딥러닝 기반 검출기의 성능이 훨씬 좋습니다.


- 딥러닝 기반 detection 방법을 조사합니다. 아래 링크를 참고하면 도움이 될 것입니다.    
  -  [How does the OpenCV deep learning face detector work?](https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/)


- opencv 는 intel cpu 을 사용할 때 dnn 모듈이 가속화를 지원하고 있습니다. 따라서 mobilenet 과 같은 작은 backbone 모델을 사용하고 ssd 를 사용한다면 충분히 만족할 만한 시간과 성능을 얻을 수 있습니다.

- hog 학습 단계에서 다양한 각도에 대한 hog 특징을 모두 추출해서 일반화 하기 어렵기 때문에 딥러닝 기반 디텍터를 사용하는 방법이 더 좋은 성능을 보입니다.   

- dlib + hog 디텍트 방식은 현재 잘 사용하지 않는 방식으로 deep learning을 사용하는 얼굴 디텍트 방식으로 2가지를 소개하겠습니다.    

1. **opencv + Haar(Haar Cascades)**   
Haar 특징기반 다단계 분류자(Feature-based Cascade Classifiers)를 이용한 물체 검출   
- **인테그랄(Integral)** 이미지를 사용해서픽셀의 합의 계산을 단순화 시켜 줍니다. 
   - 알고리즘 속도를 매우 빠르게 개선시켜 줍니다.
- **다단계 개념 사용** : 윈도우에 대한 모든 특징을 적용하는 대신, 분류자의 다른 단계로 특징을 묶고 하나씩 하나씩 적용 [윈도우가 얼굴 영역이 아닌지를 검사하는 방법]   

#### opencv + Haar(Haar Cascades) 코드  :    
``` python
import cv2, time

# load model
# haarcascade 모델을 불러와서 사용한다.   
detector = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')

# initialize video source, default 0 (webcam)
cap = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')


frame_count, tt = 0, 0

while cap.isOpened():
  ret, img = cap.read()
  if not ret:
    break

  frame_count += 1

  start_time = time.time()

  # prepare input
  result_img = img.copy()
  gray = cv2.cvtColor(result_img, cv2.COLOR_BGR2GRAY)

  # inference, find faces
  # 인테그랄(Integral) 이미지 사용을 위해 gray scale 이미지로 변환한다.  
  detections = detector.detectMultiScale(gray)

  # postprocessing
  # detection한 결과 top-left[왼쪽 위] 좌표와 rect의 너비[w], 높이[h] 값이 return된다.
  # 위의 값으로 bottom-right[오른쪽 아래] 좌표를 유추해낼 수 있다.  
  for (x1, y1, w, h) in detections:
    x2 = x1 + w
    y2 = y1 + h

    # draw rects
    cv2.rectangle(result_img, (x1, y1), (x2, y2), (255, 255, 255), 2, cv2.LINE_AA)

  # inference time
  # fps(frame pre second)별 정확도 확인
  tt += time.time() - start_time
  fps = frame_count / tt
  cv2.putText(result_img, '(haar): %.2f' % (fps), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

  # visualize
  cv2.imshow('result', result_img)
  # q 누를시 종료
  if cv2.waitKey(1) == 27:
    break


cap.release()
out.release()
cv2.destroyAllWindows()

```




2. **opencv + dnn(deep neural network)**   
pencv dnn 모듈은 directed graph 이고, 이미 만들어진 네트워크에서 순방향 실행을 위한 용도로 설계되었다.  <br>
   즉, 모델학습을 하지 않고 학습된 모델을 불러와서 사용한다.


- 딥러닝 학습은 caffe, tensorflow등의 다른 딥러닝 프레임워크에서 진행한 후 학습된 모델을 불러와 실행할 떄는 dnn 모듈을 사용한다.    


- dnn module는 Python 뿐만 아니라 c/c++ 환경에서도 동작할 수 있기에 프로그램 이식성이 높다

#### opencv + dnn(deep neural network) 코드  :    


``` python

import cv2, time

# load model
model_path = 'models/opencv_face_detector_uint8.pb'
config_path = 'models/opencv_face_detector.pbtxt'
net = cv2.dnn.readNetFromTensorflow(model_path, config_path)

conf_threshold = 0.7

# initialize video source, default 0 (webcam)
cap = cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')


frame_count, tt = 0, 0

while cap.isOpened():
  ret, img = cap.read()
  if not ret:
    break

  frame_count += 1

  start_time = time.time()

  # prepare input
  result_img = img.copy()
  h, w, _ = result_img.shape
  blob = cv2.dnn.blobFromImage(result_img, 1.0, (640, 360), [104, 117, 123], False, False)
  net.setInput(blob)

  # inference, find faces
  detections = net.forward()

  # postprocessing
    # (x1,y1) : 얼굴인식한 rect에서의 왼쪽 위 좌표
    # (x2,y2) : 얼굴인식한 rect에서의 오른쪽 아래 좌표
  for i in range(detections.shape[2]):
    confidence = detections[0, 0, i, 2]
    # conf_threshold = 0.7   : 0.7 확률 초과인 결과만 얼굴로 판단
    if confidence > conf_threshold:
      x1 = int(detections[0, 0, i, 3] * w)
      y1 = int(detections[0, 0, i, 4] * h)
      x2 = int(detections[0, 0, i, 5] * w)
      y2 = int(detections[0, 0, i, 6] * h)

      # draw rects
      # top-left corner (x1, y1) , bottom-right corner (x2, y2)  box 그리기
      cv2.rectangle(result_img, (x1, y1), (x2, y2), (255, 255, 255), int(round(h/150)), cv2.LINE_AA)
      # 위의 결과를 확률로 표시 
      cv2.putText(result_img, '%.2f%%' % (confidence * 100.), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

  # inference time
  tt += time.time() - start_time
  fps = frame_count / tt
  cv2.putText(result_img, 'FPS(dnn): %.2f' % (fps), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

  # visualize
  cv2.imshow('result', result_img)
    # esc 누를시 종료
  if cv2.waitKey(1) == 27:
    break


cap.release()
out.release()
cv2.destroyAllWindows()

```

---

#### 5. 인원 수, 각도 등 각 문제에 대해서 1 - 4번을 반복해주세요. (정해진 답은 없습니다.)
자유롭게 설계해주세요. 각도 문제에 대해서는 아래 방법을 적용해볼 수 있습니다.

[Facial Landmark Detection](https://learnopencv.com/facial-landmark-detection/)

---

##  5. 칼만 필터 적용하기
**1. 카메라 앞에서 가만히 있을 때 스티커의 움직임에서 보이는 문제**      
가만히 있어도 스티커의 크리가 일정하게 유지되지 않고, 떨리는 것처럼 보이는 현상이 발생합니다.


**2. 이론 강의에서 배운 칼만 필터를 적용해서 스티커 움직임을 안정화시켜 주세요.**   
칼만 필터를 구현한 모듈인 kalman.py와 이를 이용하여 tracker를 구현한 kpkf.py를 보고  wecam_sticker.py에 칼만필터를 적용해봅시다 

- 동영상에 칼만필터를 적용한 결과 더 부드러운 스티커 이미지가 보여집니다.   

#### 1. 카메라 앞에서 가만히 있을 때 스티커의 움직임을 관찰해 주세요. 어떤 문제가 발생하나요?
가만히 있어도 스티커의 크리가 일정하게 유지되지 않고, 떨리는 것처럼 보이는 현상이 발생합니다.

**2. 이론 강의에서 배운 칼만 필터를 적용해서 스티커 움직임을 안정화시켜 주세요.**  

동영상에 칼만필터를 적용한 결과 더 부드러운 스티커 이미지를 보입니다.



**dlib + hog가 아닌 openCV_dnn을 적용해서 사용**   

- 얼굴을 인식하는 bounding box의 (w,h) 크기 비례해서 왕관 이미지를 띄워서 얼굴의 각도에 따른 얼굴의 크기에 반응을 한다.  

- 거리에 따라서 box의 크기가 변동하기에 왕관의 크기도 변동된다.  


> 좀 더 자연스러운 이미지 출력 + dlib_hog보다 빠른 속도

---


#### custom webcam_sticker.py

``` python
import numpy as np
import cv2
import dlib

from new_addsticker_test import img2sticker

#detector_hog = dlib.get_frontal_face_detector()
#landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

# opencv + dnn caffe
model_path = 'models/res10_300x300_ssd_iter_140000.caffemodel'
config_path = 'models/deploy.prototxt'
net = cv2.dnn.readNetFromCaffe(config_path, model_path)
#  opencv + dnn  -no caffe
#model_path = 'models/opencv_face_detector_uint8.pb'
#config_path = 'models/opencv_face_detector.pbtxt'
#net = cv2.dnn.readNetFromTensorflow(model_path, config_path)

# opencv haarcase model
#detector = cv2.CascadeClassifier('models/haarcascade_frontalface_default.xml')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)
    # 컴퓨터와 연결된 웹캠 사용시 아래 
    vc = cv2.VideoCapture(0) 
    # 스마트폰 어플리케이션을 웹캠과 같이 rtmp로 스트리밍해서 사용
    #vc = cv2.VideoCapture('rtmp://rtmp.streamaxia.com/streamaxia/5467136366cf811e')
    img_sticker = cv2.imread('./images/king.png')
    #sticker_img = img_sticker.copy()
    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while vc.isOpened():
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전


        # prepare input
        result_img = img.copy()
        h, w, _ = result_img.shape
        print("webcam : ", result_img.shape)
        blob = cv2.dnn.blobFromImage(result_img, 1.0, (360, 300), [104, 117, 123], False, False)
        net.setInput(blob)

        # inference, find faces
        detections = net.forward()  

        # 스티커 메소드를 사용  
        
        x1 = y1 = x2 = y2 = 0
        confidence = 0
        conf_threshold = 0.8
        box_w, box_h =0,0
        print("h,w,_ : ", h,w)
        # postprocessing
        for i in range(detections.shape[2]):

            confidence = detections[0, 0, i, 2]

            if confidence > conf_threshold:
                x1 = int(detections[0, 0, i, 3] * w)
                y1 = int(detections[0, 0, i, 4] * h)
                x2 = int(detections[0, 0, i, 5] * w)
                y2 = int(detections[0, 0, i, 6] * h)
                box_w =  x2 - x1
                box_h =  y2 - y1
                cv2.rectangle(result_img, (x1, y1), (x2, y2), (255, 255, 255), int(round(h/150)), cv2.LINE_AA)
                cv2.putText(result_img, '%.2f%%' % (confidence * 100.), (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                #cv2.putText(result_img, f'w : {box_w}  h : {box_h}'   , (x2, y2-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)

                break
         # sticker
        # 낮은 정확도의 경우 얼굴이 아닐 수 있으므로 스티커를 붙이지 않도록 한다.
        # confidence < conf_threshold 일떄
        # box_w, box_h의 값이 0이므로 sticker_img의 크기가 (0,0)이 되는 오류 방지
        if confidence > conf_threshold:
            #print(f"box_w {box_w}  , box_h : {box_h}")
            #print(f"img_sticker :  {img_sticker.shape}")
  
            try:   # 예외가 발생할때 except가 출력지만, 프로그램 종료가 되지 않도록 한다.
                sticker_img = img_sticker.copy()
                refined_x = x1+box_w
                refined_y = y1-box_h
                
                # 스티커 이미지가 화면 경계 밖으로 나가는 경우 크기 조정
                if refined_y < 0 :
                    box_h = y1
                
                if refined_x > w :
                    box_w = w-x1

                sticker_img = cv2.resize(sticker_img, dsize=(box_w,box_h))
                sticker_img_alpha = 1.0
                background_alpha = 1.0 
                
                result_img[y1-box_h:y1, x1:x1+box_w] = \
                cv2.addWeighted( sticker_img[:, :, :3], sticker_img_alpha  , result_img[y1-box_h:y1, x1:x1+box_w] , background_alpha ,0)
            except: 
                # 이미지 크기 문제 [cv2.addWeighted]에서 오류가 발생하기 쉬우니 이미지 크기들을 출력해서 확인한다. 
                print("\n======================\n image blending except \n======================\n")
                print("sticker_img : ", sticker_img.shape )
                print("sticker_area : ", result_img[y1:y1-box_h, x1:x1+box_w].shape )
                                                  
        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)
        
        cv2.imshow('show', result_img)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()
```

---

## 회고

- 스노우 카메라를 썼을때는 실시간 detection 중 스티커 붙이기가 그렇게 어렵지 않을 줄 알았는데 pixel 값을 계산하면서 1 px 차이로 오류가 발생하거나 고민끝에 생각한 픽셀 범위의 경우 깔끔하게 스티커가 붙여지지 않는 문제들을 겪으면서, 쉬운 일은 없다는 것을 다시 깨달은것 같습니다.   


- 이번 프로젝트는 딥러닝에 대한 이해보다는 파이썬에서 실시간 이미지를 다룰때 생각해야 하는 픽셀들의 범위에 대한 이해가 더 필요했던것 같습니다.   
마지막 wecam_sticker_CV_dnn.py를 만들때 dnn 모듈의 경우 사용하는데 긴 시간이 걸리지 않았지만, 픽셀 값 계산이 잘안되어서 범위를 지정하는데 더 많은 시간이 걸렸습니다.   


- 실시간 이미지 처리에 대해 배운 시간인것 같습니다.   
